In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [9]:
import pandas as pd
import numpy as np


In [16]:
df = pd.read_csv('../data/df1.csv').drop('Unnamed: 0', axis=1)
df

,Date,Open,High,Low,Close,Volume,MA10,MA50,EMA20,Return,Log Return,Volatility,Distance_MA50,Distance_EMA20,Cumulated_Return_5d,RSI14,Trend
0,2010-01-04 00:00:00-05:00,84.307727,85.072999,83.662495,85.027985,118944600,NaN,NaN,85.027985,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,Bear
1,2010-01-05 00:00:00-05:00,84.975448,85.290560,84.667835,85.253044,111579900,NaN,NaN,85.146141,0.002647,0.002643,NaN,NaN,0.001256,NaN,NaN,Bear
2,2010-01-06 00:00:00-05:00,85.170489,85.523116,85.102967,85.313042,116074400,NaN,NaN,85.207426,0.000704,0.000704,NaN,NaN,0.001240,NaN,NaN,Bear
3,2010-01-07 00:00:00-05:00,85.155492,85.778217,84.915407,85.673180,131091100,NaN,NaN,85.341882,0.004221,0.004212,NaN,NaN,0.003882,NaN,NaN,Bear
4,2010-01-08 00:00:00-05:00,85.448115,85.995813,85.275556,85.958298,126402800,NaN,NaN,85.490988,0.003328,0.003322,NaN,NaN,0.005466,NaN,NaN,Bear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3768,2024-12-23 00:00:00-05:00,584.031655,588.390442,580.839104,587.787537,57635800,590.267548,582.895714,589.035274,0.005988,0.005970,0.008463,0.008392,-0.002118,-0.016642,43.149005,Range
3769,2024-12-24 00:00:00-05:00,589.141630,594.360375,588.558452,594.320801,33160100,590.319147,583.270117,589.538658,0.011115,0.011054,0.008789,0.018946,0.008112,-0.001599,46.098014,Range
3770,2024-12-26 00:00:00-05:00,592.541743,595.487135,591.138241,594.360413,41219100,589.915662,583.734758,589.997872,0.000067,0.000067,0.008719,0.018203,0.007394,0.029140,46.981957,Range
3771,2024-12-27 00:00:00-05:00,590.604424,590.841689,583.903150,588.103821,64969300,589.194843,584.024619,589.817487,-0.010527,-0.010582,0.009024,0.006985,-0.002905,0.018619,40.972868,Range
